In [19]:
import pandas as pd
import altair as alt
import eco_style
alt.themes.enable('light')
import numpy as np

In [118]:
df = pd.read_excel("labourproductivityitls (1).xls", sheet_name="B4", skiprows=4)
df = df.query("ITL_level == 'ITL2'")
df = df[['ITL_code', 'Region_name', 'Pounds_2022']]
df.columns= ['ITL_code', 'name', 'gva_pj']
df['parent'] = df['ITL_code'].str[:3]

df['jitter'] = np.random.rand(len(df)) * 0.00000


gva_df = df.copy()

df = pd.read_excel("labourproductivityitls (1).xls", sheet_name="Productivity Jobs", skiprows=4)
df = df.query("ITL_level == 'ITL2'")
df = df[['ITL_code', 'Region_name', 'Jobs_2022']]
df.columns= ['ITL_code', 'name', 'jobs']
jobs_df = df.copy()

df = pd.read_excel("labourproductivityitls (1).xls", sheet_name="Productivity Jobs", skiprows=4)
df = df[['ITL_code', 'Region_name']]
df.columns= ['parent', 'parent_name']
df = df[df.parent.isin(gva_df.parent.unique())]
parents_df = df.copy()

df = pd.merge(gva_df, jobs_df, on=['ITL_code', 'name'])
df = pd.merge(df, parents_df, on='parent')

# get the max value for each parent
df['parent_max'] = df.groupby('parent')['gva_pj'].transform('max')
df['parent_min'] = df.groupby('parent')['gva_pj'].transform('min')

# label the highest and lowest values for each parent
df['label_high'] = np.where(df['gva_pj'] == df['parent_max'], df.name, '')
df['label_low'] = np.where(df['gva_pj'] == df['parent_min'], df.name, '')

discretionary_labels = [
    "Inner London - West",
    "Greater Manchester",
    "Outer London - East and North East",
    "West Midlands",
    "West Wales and The Valleys",
    "Gloucestershire, Wiltshire and Bath/Bristol Area",
]

df['label'] = np.where(df['name'].isin(discretionary_labels), df['name'], '')

chart = alt.Chart(df).mark_circle(
    color="#36B7B4",
    opacity=0.6,
    stroke='black',
    strokeWidth=0.5
).encode(
    y=alt.Y("parent_name:N", title="", sort=alt.EncodingSortField(field="parent_max", order='descending')),
    yOffset=alt.Y("jitter:Q", title=""),
    x=alt.X("gva_pj:Q", title="", axis=alt.Axis(labelExpr="'£' + format(datum.value, ',.0f')")),
    size=alt.Size("jobs:Q", 
                  legend=alt.Legend(title="Jobs", orient='top', titleOrient='left', titleFontWeight='normal', titleFont="Circular Std", titleColor="#676A86"),
                  title="Jobs", scale=alt.Scale(range=[0, 900])),
    tooltip=['name', 'gva_pj', 'jobs']
)

# add a rule for the parent
rule = alt.Chart(df).mark_rule(
    strokeDash=[3, 4],
    color='#676A86').encode(
    y=alt.Y("parent_name:N", title="", sort=alt.EncodingSortField(field="parent_max", order='descending')),
)

labels_high = alt.Chart(df).mark_text(
    align='left',
    baseline='middle',
    dx=5,
    font='Circular Std',
    fontSize=10,
    color='#676A86',
    fontWeight='normal'
).encode(
    y=alt.Y("parent_name:N", title="", sort=alt.EncodingSortField(field="parent_max", order='descending')),
    x=alt.X("gva_pj:Q"),
    text=alt.Text("label_high:N"),
)

labels_low = alt.Chart(df).mark_text(
    align='right',
    baseline='middle',
    dx=-5,
    font='Circular Std',
    fontSize=10,
    color='#676A86',
    fontWeight='normal'
).encode(
    y=alt.Y("parent_name:N", title="", sort=alt.EncodingSortField(field="parent_max", order='descending')),
    x=alt.X("gva_pj:Q"),
    text=alt.Text("label_low:N"),
)

labels = alt.Chart(df).mark_text(
    align='center',
    baseline='middle',
    dx=0,
    dy=-20,

    font='Circular Std',
    fontSize=10,
    color='#676A86',
    fontWeight='normal'
).encode(
    y=alt.Y("parent_name:N", title="", sort=alt.EncodingSortField(field="parent_max", order='descending')),
    x=alt.X("gva_pj:Q"),
    text=alt.Text("label:N"),
)

chart = chart + labels #labels_high + labels_low + rule


chart = chart.properties(
    width=400,
    height=500
)

chart.save("gva_ineq.png", scale_factor=3.0)
chart.save("gva_ineq.json")

chart



/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/

alt.LayerChart(...)

In [121]:
df.parent_name.unique()

array(['North East', 'North West', 'Yorkshire and The Humber',
       'East Midlands', 'West Midlands', 'East', 'London', 'South East',
       'South West', 'Wales', 'Scotland', 'Northern Ireland'],
      dtype=object)

## Another Approach

In [217]:
df = pd.read_excel("labourproductivitycityregions.xls", sheet_name="B4", skiprows=4)
df = df[["Area_Code", "City_Region", "Pounds_2022"]]
df.columns = ["code", "name", "gva_pj"]
cities_gva_df = df.copy()

df = pd.read_excel("labourproductivitycityregions.xls", sheet_name="Productivity Jobs", skiprows=4)
df = df[["Area_Code", "City_Region", "Jobs_2022"]]
df.columns = ["code", "name", "jobs"]
cities_jobs_df = df.copy()

cities_df = pd.merge(cities_gva_df, cities_jobs_df, on=['code', 'name']).dropna(subset=['gva_pj'])

# exclude the city regions that aren't really city regions
cities_df = cities_df[~cities_df.name.isin([
    'Causeway Coast and Glens Growth Deal',
    'Ayrshires',
    'Highlands and Islands',
    'Greater London Authority',
    'Scottish Island Councils',
    'South of Scotland',
    'Growing Mid Wales',
    'North Wales Economic Ambition Board',
])]

short_names = {
  "Greater Manchester Combined Authority": "Greater Manchester",
  "South Yorkshire Mayoral Combined Authority": "South Yorkshire",
  "West Yorkshire Combined Authority": "West Yorkshire",
  "Liverpool City Region Combined Authority": "Liverpool",
  "Tees Valley Combined Authority": "Tees Valley",
  "West Midlands Combined Authority": "West Midlands",
  "Cambridgeshire and Peterborough Combined Authority": "Cambridgeshire & Peterborough",
  "West of England Combined Authority": "West of England",
  "North East Combined Authority": "North East",
  "North of Tyne Combined Authority": "North of Tyne",
  "Greater London Authority": "Greater London",
  "Belfast City Region": "Belfast",
  "Derry-Londonderry City Region": "Derry-Londonderry",
  "Aberdeen City Region": "Aberdeen",
  "Edinburgh and South East Scotland City Region": "Edinburgh & SE Scot",
  "Forth Valley Region": "Forth Valley",
  "Glasgow City Region": "Glasgow",
  "Greater Brighton City Region": "Greater Brighton",
  "Inner London": "Inner London",
  "Mid South West Growth Deal": "Mid South West",
  "Outer London": "Outer London",
  "Tay Cities Region": "Tay Cities",
  "Cardiff Capital Region": "Cardiff",
  "Swansea Bay City Region": "Swansea Bay"
}

exclude_labels = [
    "Cambridgeshire and Peterborough Combined Authority",
    "Swansea Bay City Region",
    "North of Tyne Combined Authority",
    "Tay Cities Region",
    "Belfast City Region",
]

cities_df['short_name'] = cities_df['name'].map(short_names)
cities_df['type'] = "City Region"

cities_regions = {
  "Greater Manchester Combined Authority": "North West",
  "South Yorkshire Mayoral Combined Authority": "Yorkshire and The Humber",
  "West Yorkshire Combined Authority": "Yorkshire and The Humber",
  "Liverpool City Region Combined Authority": "North West",
  "Tees Valley Combined Authority": "North East",
  "West Midlands Combined Authority": "West Midlands",
  "Cambridgeshire and Peterborough Combined Authority": "East",
  "West of England Combined Authority": "South West",
  "North East Combined Authority": "North East",
  "North of Tyne Combined Authority": "North East",
  "Greater London Authority": "London",
  "Belfast City Region": "Northern Ireland",
  "Derry-Londonderry City Region": "Northern Ireland",
  "Aberdeen City Region": "Scotland",
  "Ayrshires": "Scotland",
  "Causeway Coast and Glens Growth Deal": "Northern Ireland",
  "Edinburgh and South East Scotland City Region": "Scotland",
  "Forth Valley Region": "Scotland",
  "Glasgow City Region": "Scotland",
  "Greater Brighton City Region": "South East",
  "Highlands and Islands": "Scotland",
  "Inner London": "London",
  "Mid South West Growth Deal": "Northern Ireland",
  "Outer London": "London",
  "Scottish Island Councils": "Scotland",
  "South of Scotland": "Scotland",
  "Tay Cities Region": "Scotland",
  "Cardiff Capital Region": "Wales",
  "Growing Mid Wales": "Wales",
  "North Wales Economic Ambition Board": "Wales",
  "Swansea Bay City Region": "Wales"
}

cities_df['region_name'] = cities_df['name'].map(cities_regions)

df = pd.read_excel("labourproductivitycityregions.xls", sheet_name="Geography Lookup", skiprows=0)
# LAD23CD	LAD23NM	CRCD	CRNM	Country
df.columns = ['code', 'name', 'city_code', 'city_name', 'country']
df = df[df.city_name.isin(cities_df.name.unique())]
lads_to_exclude = df.code.unique()

df = pd.read_excel("labourproductivitylocalauthorities.xls", sheet_name="B3", skiprows=4)
df = df[['LAD_Code', 'LAD_Name', 'Pounds_2022']]
df.columns = ['code', 'name', 'gva_pj']
lad_gva_df = df.copy()

df = pd.read_excel("labourproductivitylocalauthorities.xls", sheet_name="Productivity Jobs", skiprows=4)
df = df[['LAD_Code', 'LAD_Name', 'Jobs_2022']]
df.columns = ['code', 'name', 'jobs']
lad_jobs_df = df.copy()

lads_df = pd.merge(lad_gva_df, lad_jobs_df, on=['code', 'name']).dropna(subset=['gva_pj'])
lads_df = lads_df[~lads_df.code.isin(lads_to_exclude)]

df = pd.read_csv("LADs_to_region.csv")
df = df[['LAD23CD', 'RGN23CD', 'RGN23NM']]
df.columns = ['code', 'region_code', 'region_name']

lads_df = pd.merge(lads_df, df, on='code', how='left')
lads_df[lads_df.name.astype(str).str.contains("Causeway Coast and Glens")]

# for the non-England regions, we can just use the first letter of the code
lads_df.loc[lads_df.code.str.startswith("S"), 'region_code'] = "S92000003"
lads_df.loc[lads_df.code.str.startswith("W"), 'region_code'] = "W92000004"
lads_df.loc[lads_df.code.str.startswith("N"), 'region_code'] = "N92000002"
lads_df.loc[lads_df.code.str.startswith("S"), 'region_name'] = "Scotland"
lads_df.loc[lads_df.code.str.startswith("W"), 'region_name'] = "Wales"
lads_df.loc[lads_df.code.str.startswith("N"), 'region_name'] = "Northern Ireland"

lads_df['type'] = "Local Authority"

df = pd.concat([cities_df, lads_df])

# fill the NaN region_code s with the code
region_codes = df.dropna(subset=['region_code']).groupby('region_name')['region_code'].first().to_dict()
df['region_code'] = df['region_code'].fillna(df['region_name'].map(region_codes))

region_codes = {
  "North East": "E12000001",
  "North West": "E12000002",
  "Yorkshire and The Humber": "E12000003",
  "East Midlands": "E12000004",
  "West Midlands": "E12000005",
  "East": "E12000006",
  "London": "E12000007",
  "South East": "E12000008",
  "South West": "E12000009",
  "Wales": "W92000004",
  "Scotland": "S92000003",
  "Northern Ireland": "N92000002"
}

df['region_code'] = df['region_name'].map(region_codes)

df = df.rename(columns={"region_name": "parent_name", "region_code": "parent"})

df['parent_max'] = df.groupby('parent')['gva_pj'].transform('max')
df['parent_median'] = df.groupby('parent')['gva_pj'].transform('median')

df['parent_city_max'] = df.query("type == 'City Region'").groupby('parent')['gva_pj'].transform('max')
df['parent_city_min'] = df.query("type == 'City Region'").groupby('parent')['gva_pj'].transform('min')

# label the highest and lowest city region for each parent
df['label_high'] = np.where(df['gva_pj'] == df['parent_city_max'], df.short_name, '')
df['label_low'] = np.where(df['gva_pj'] == df['parent_city_min'], df.short_name, '')
# exclude the low when it's the same as the high
df['label_low'] = np.where(df['label_low'] == df['label_high'], '', df['label_low'])

# exclude some labels
df['label_low'] = np.where(df['name'].isin(exclude_labels), '', df['label_low'])
df['label_high'] = np.where(df['name'].isin(exclude_labels), '', df['label_high'])

df = df.drop(columns=['parent_city_max', 'parent_city_min'])

In [220]:
chart = alt.Chart(df).mark_circle(
    color="#36B7B4",
    opacity=0.6,
    stroke='black',
    strokeWidth=0.5
).encode(
    y=alt.Y("parent_name:N", title="", sort=alt.EncodingSortField(field="parent_median", order='descending')),
    x=alt.X("gva_pj:Q", title="", axis=alt.Axis(labelExpr="'£' + format(datum.value, ',.0f')")),
    size=alt.Size("jobs:Q", 
                  legend=alt.Legend(title="", 
                                    direction='horizontal',
                                    legendY=-50,
                                    labelExpr="format(datum.value, ',.0s') + (datum.value == 1000000 ? ' jobs' : '')",
                                    orient='none', titleOrient='left', titleFontWeight='normal', titleFont="Circular Std", titleColor="#676A86"),
                  title="Jobs", scale=alt.Scale(
                      type='linear',
                      zero=False,
                      range=[50, 900])),
    color=alt.Color("type:N", legend=alt.Legend(
        title="", orient='none', 
        legendY=-15,
        direction='horizontal',
         
          titleOrient='left', titleFontWeight='normal', titleFont="Circular Std", titleColor="#676A86"
    ),
                scale=alt.Scale(domain=['Local Authority', 'City Region'], range=['#36B7B4', '#E6224B'])),
    tooltip=['name', 'gva_pj', 'jobs', 'type'],
    order=alt.Order("type", sort="descending")
)

# add a rule for the parent
rule = alt.Chart(df).mark_rule(
    strokeDash=[3, 4],
    color='#676A86').encode(
    y=alt.Y("parent_name:N", title="", sort=alt.EncodingSortField(field="parent_median", order='descending')),
)

labels_high = alt.Chart(df).mark_text(
    align='left',
    baseline='middle',
    dx=5,
    dy=-15,
    font='Circular Std',
    fontSize=10,
    color='#E6224B',
    fontWeight='normal'
).encode(
    y=alt.Y("parent_name:N", title="", sort=alt.EncodingSortField(field="parent_median", order='descending')),
    x=alt.X("gva_pj:Q"),
    text=alt.Text("label_high:N"),
)

labels_low = alt.Chart(df).mark_text(
    align='right',
    baseline='middle',
    dx=-5,
    dy=-15,
    font='Circular Std',
    fontSize=10,
    color='#E6224B',
    fontWeight='normal'
).encode(
    y=alt.Y("parent_name:N", title="", sort=alt.EncodingSortField(field="parent_median", order='descending')),
    x=alt.X("gva_pj:Q"),
    text=alt.Text("label_low:N"),
)

labels = alt.Chart(df).mark_text(
    align='center',
    baseline='middle',
    dx=0,
    dy=-20,

    font='Circular Std',
    fontSize=10,
    color='#676A86',
    fontWeight='normal'
).encode(
    y=alt.Y("parent_name:N", title="", sort=alt.EncodingSortField(field="parent_median", order='descending')),
    x=alt.X("gva_pj:Q"),
    text=alt.Text("label:N"),
)

chart = chart  + labels_high + labels_low 


chart = chart.properties(
    width=400,
    height=500
)

chart.save("gva_lads_cities.png", scale_factor=3.0)
chart.save("gva_lads_cities.json")

chart

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/

alt.LayerChart(...)

In [155]:
df.region_name.unique()

array(['North West', 'Yorkshire and The Humber', 'North East',
       'West Midlands', 'East', 'South West', 'London',
       'Northern Ireland', 'Scotland', 'South East', 'Wales',
       'East Midlands', 'East of England'], dtype=object)

In [149]:
lads_df

,code,name,gva_pj,jobs,region_code,region_name
0,E06000007,Warrington,60988.0,143181.0,E12000002,North West
1,E06000008,Blackburn with Darwen,48570.5,73598.0,E12000002,North West
2,E06000009,Blackpool,42587.0,67218.0,E12000002,North West
3,E06000010,"Kingston upon Hull, City of",49078.1,131247.0,E12000003,Yorkshire and The Humber
4,E06000011,East Riding of Yorkshire,59087.7,140831.0,E12000003,Yorkshire and The Humber
...,...,...,...,...,...,...
217,W06000004,Denbighshire,44893.0,46015.0,NaN,NaN
218,W06000005,Flintshire,59065.1,75063.0,NaN,NaN
219,W06000006,Wrexham,56226.9,63474.0,NaN,NaN
220,W06000008,Ceredigion,42034.1,34204.0,NaN,NaN


In [143]:
cities_df.name.unique()

array(['Greater Manchester Combined Authority',
       'South Yorkshire Mayoral Combined Authority',
       'West Yorkshire Combined Authority',
       'Liverpool City Region Combined Authority',
       'Tees Valley Combined Authority',
       'West Midlands Combined Authority',
       'Cambridgeshire and Peterborough Combined Authority',
       'West of England Combined Authority',
       'North East Combined Authority',
       'North of Tyne Combined Authority', 'Greater London Authority',
       'Belfast City Region', 'Derry-Londonderry City Region',
       'Aberdeen City Region', 'Ayrshires',
       'Causeway Coast and Glens Growth Deal',
       'Edinburgh and South East Scotland City Region',
       'Forth Valley Region', 'Glasgow City Region',
       'Greater Brighton City Region', 'Highlands and Islands',
       'Inner London', 'Mid South West Growth Deal', 'Outer London',
       'Scottish Island Councils', 'South of Scotland',
       'Tay Cities Region', 'Cardiff Capital Region

In [136]:
df = pd.read_excel("labourproductivitylocalauthorities.xls", sheet_name="B3", skiprows=4)
df = df[['LAD_Code', 'LAD_Name', 'Pounds_2022']]
df.columns = ['code', 'name', 'gva_pj']
lad_gva_df = df.copy()

df = pd.read_excel("labourproductivitylocalauthorities.xls", sheet_name="Productivity Jobs", skiprows=4)
df = df[['LAD_Code', 'LAD_Name', 'Jobs_2022']]
df.columns = ['code', 'name', 'jobs']
lad_jobs_df = df.copy()

lads_df = pd.merge(lad_gva_df, lad_jobs_df, on=['code', 'name']).dropna(subset=['gva_pj'])


lads_df[lads_df.name.astype(str).str.contains("Belfast")]


,code,name,gva_pj,jobs
298,N09000003,Belfast,61868.3,246532.0


In [130]:
lads_to_exclude

array(['E09000001', 'E09000002', 'E09000003', 'E09000004', 'E09000005',
       'E09000006', 'E09000007', 'E09000008', 'E09000009', 'E09000010',
       'E09000011', 'E09000012', 'E09000013', 'E09000014', 'E09000015',
       'E09000016', 'E09000017', 'E09000018', 'E09000019', 'E09000020',
       'E09000021', 'E09000022', 'E09000023', 'E09000024', 'E09000025',
       'E09000026', 'E09000027', 'E09000028', 'E09000029', 'E09000030',
       'E09000031', 'E09000032', 'E09000033', 'E08000001', 'E08000002',
       'E08000003', 'E08000004', 'E08000005', 'E08000006', 'E08000007',
       'E08000008', 'E08000009', 'E08000010', 'E08000016', 'E08000017',
       'E08000018', 'E08000019', 'E08000032', 'E08000033', 'E08000034',
       'E08000035', 'E08000036', 'E06000006', 'E08000011', 'E08000012',
       'E08000013', 'E08000014', 'E08000015', 'E06000005', 'E06000001',
       'E06000002', 'E06000003', 'E06000004', 'E08000025', 'E08000026',
       'E08000027', 'E08000028', 'E08000029', 'E08000030', 'E080

In [129]:
lads_df[lads_df.name.str.contains("Belfast")]

,code,name,gva_pj,jobs


In [ ]:
{
  "Greater Manchester Combined Authority": "North West",
  "South Yorkshire Mayoral Combined Authority": "Yorkshire and The Humber",
  "West Yorkshire Combined Authority": "Yorkshire and The Humber",
  "Liverpool City Region Combined Authority": "North West",
  "Tees Valley Combined Authority": "North East",
  "West Midlands Combined Authority": "West Midlands",
  "Cambridgeshire and Peterborough Combined Authority": "East",
  "West of England Combined Authority": "South West",
  "North East Combined Authority": "North East",
  "North of Tyne Combined Authority": "North East",
  "Greater London Authority": "London",
  "Belfast City Region": "Northern Ireland",
  "Derry-Londonderry City Region": "Northern Ireland",
  "Aberdeen City Region": "Scotland",
  "Ayrshires": "Scotland",
  "Causeway Coast and Glens Growth Deal": "Northern Ireland",
  "Edinburgh and South East Scotland City Region": "Scotland",
  "Forth Valley Region": "Scotland",
  "Glasgow City Region": "Scotland",
  "Greater Brighton City Region": "South East",
  "Highlands and Islands": "Scotland",
  "Inner London": "London",
  "Mid South West Growth Deal": "Northern Ireland",
  "Outer London": "London",
  "Scottish Island Councils": "Scotland",
  "South of Scotland": "Scotland",
  "Tay Cities Region": "Scotland",
  "Cardiff Capital Region": "Wales",
  "Growing Mid Wales": "Wales",
  "North Wales Economic Ambition Board": "Wales",
  "Swansea Bay City Region": "Wales"
}

In [116]:
cities_df.name.unique()

array(['Greater Manchester Combined Authority',
       'South Yorkshire Mayoral Combined Authority',
       'West Yorkshire Combined Authority',
       'Liverpool City Region Combined Authority',
       'Tees Valley Combined Authority',
       'West Midlands Combined Authority',
       'Cambridgeshire and Peterborough Combined Authority',
       'West of England Combined Authority',
       'North East Combined Authority',
       'North of Tyne Combined Authority', 'Greater London Authority',
       'Belfast City Region', 'Derry-Londonderry City Region',
       'Aberdeen City Region', 'Ayrshires',
       'Causeway Coast and Glens Growth Deal',
       'Edinburgh and South East Scotland City Region',
       'Forth Valley Region', 'Glasgow City Region',
       'Greater Brighton City Region', 'Highlands and Islands',
       'Inner London', 'Mid South West Growth Deal', 'Outer London',
       'Scottish Island Councils', 'South of Scotland',
       'Tay Cities Region', 'Cardiff Capital Region

# Prod

In [91]:
df = pd.read_excel("subnatindicators.xlsx", sheet_name="GVA per hour", skiprows=6)
df = df.dropna(subset=['Local Authority District', 'County or Unitary Authority'], how='all')
df = df[["Area Code", "Local Authority District", 'County or Unitary Authority',"Gross Value Added (GVA) per hour worked (£)"]]
df.columns = ['id', 'name', 'County or Unitary Authority', 'value']
df['name'] = df.name.fillna(df['County or Unitary Authority'])

df = df[['id', 'name', 'value']]

df.to_csv("gva_per_hour_LAD.csv", index=False)

df

,id,name,value
0,E06000047,County Durham,30.64
1,E06000005,Darlington,28.56
2,E06000001,Hartlepool,29.84
3,E06000002,Middlesbrough,29.50
4,E06000057,Northumberland,30.43
...,...,...,...
369,N09000007,Lisburn and Castlereagh,33.68
370,N09000008,Mid and East Antrim,34.77
371,N09000009,Mid Ulster,37.21
372,N09000010,"Newry, Mourne and Down",31.13


# Income

In [95]:
df

,Unnamed: 0,Counties,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,Regions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,"Nations (England, Wales, Scotland, Northern Ir...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,"Countries (UK, GB, England and Wales)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Area Code,Country,Nation,Region,County or Unitary Authority,Local Authority District,Gross median weekly pay (£),Lower 95% Confidence Interval,Upper 95% Confidence Interval,Notes
4,K02000001,United Kingdom,NaN,NaN,NaN,NaN,574.9,573.773196,576.026804,NaN
...,...,...,...,...,...,...,...,...,...,...
406,N09000007,NaN,NaN,NaN,NaN,Lisburn and Castlereagh,508.3,na,na,NaN
407,N09000008,NaN,NaN,NaN,NaN,Mid and East Antrim,487.3,na,na,NaN
408,N09000009,NaN,NaN,NaN,NaN,Mid Ulster,510.8,na,na,NaN
409,N09000010,NaN,NaN,NaN,NaN,"Newry, Mourne and Down",481.6,na,na,NaN


In [101]:
df = pd.read_excel("subnatindicators.xlsx", sheet_name="Weekly pay", skiprows=10)
df = df.dropna(subset=['Local Authority District', 'County or Unitary Authority'], how='all')
df = df[["Area Code", "Local Authority District", 'County or Unitary Authority',"Gross median weekly pay (£)"]]
df.columns = ['id', 'name', 'County or Unitary Authority', 'value']
df['name'] = df.name.fillna(df['County or Unitary Authority'])

df = df[['id', 'name', 'value']]

df.to_csv("weekly_pay_LAD.csv", index=False)
df

,id,name,value
5,E06000047,County Durham,531.9
6,E06000005,Darlington,530.5
7,E06000001,Hartlepool,521.2
8,E06000002,Middlesbrough,481.9
9,E06000057,Northumberland,543.6
...,...,...,...
402,N09000007,Lisburn and Castlereagh,508.3
403,N09000008,Mid and East Antrim,487.3
404,N09000009,Mid Ulster,510.8
405,N09000010,"Newry, Mourne and Down",481.6


In [97]:
df

,Area Code,Country,Nation,Region,County or Unitary Authority,Local Authority District,Gross median weekly pay (£),Lower 95% Confidence Interval,Upper 95% Confidence Interval,Notes
5,E06000047,NaN,NaN,NaN,County Durham,NaN,531.9,503.751852,560.048148,NaN
6,E06000005,NaN,NaN,NaN,Darlington,NaN,530.5,464.99386,596.00614,NaN
7,E06000001,NaN,NaN,NaN,Hartlepool,NaN,521.2,450.712912,591.687088,NaN
8,E06000002,NaN,NaN,NaN,Middlesbrough,NaN,481.9,435.618324,528.181676,NaN
9,E06000057,NaN,NaN,NaN,Northumberland,NaN,543.6,508.439952,578.760048,NaN
...,...,...,...,...,...,...,...,...,...,...
402,N09000007,NaN,NaN,NaN,NaN,Lisburn and Castlereagh,508.3,na,na,NaN
403,N09000008,NaN,NaN,NaN,NaN,Mid and East Antrim,487.3,na,na,NaN
404,N09000009,NaN,NaN,NaN,NaN,Mid Ulster,510.8,na,na,NaN
405,N09000010,NaN,NaN,NaN,NaN,"Newry, Mourne and Down",481.6,na,na,NaN


In [93]:
df.value.quantile([0.01, 0.99])

0.01    25.2038
0.99    71.1335
Name: value, dtype: float64